In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from seaborn import color_palette
import cv2

In [ ]:
def upsample(inputs, out_shape,format):
    """Upsamples to `out_shape` using nearest neighbor interpolation."""
    if format != 'channels_first':
        n_h = out_shape[2]
        n_w = out_shape[1]
    else:
        inputs = tf.transpose(inputs, [0, 2, 3, 1])
        n_h = out_shape[3]
        n_w = out_shape[2]
    inputs = tf.image.resize_nearest_neighbor(inputs, (n_h, n_w))

    if format == 'channels_first':
        inputs = tf.transpose(inputs, [0, 3, 1, 2])

    return inputs

In [ ]:
def build_boxes(inputs):
    """Computes top left and bottom right points of the boxes."""
    center_x, center_y, width, height, confidence, classes = \
        tf.split(inputs, [1, 1, 1, 1, 1, -1], axis=-1) # negative value to automatically determine the size of the remaining part

    tleft_x = center_x - width / 2
    tleft_y = center_y - height / 2
    lright_x = center_x + width / 2
    lright_y = center_y + height / 2

    boxes = tf.concat([tleft_x, tleft_y,
                       lright_x, lright_y,
                       confidence, classes], axis=-1)

    return boxes


def non_max_suppression(inputs, n_classes, max_output_size, iou_thres,c_thresh):
    batch = tf.unstack(inputs)
    boxes_dicts = []
    for boxes in batch:
        boxes = tf.boolean_mask(boxes, boxes[:, 4] > c_thresh)
        classes = tf.argmax(boxes[:, 5:], axis=-1)
        classes = tf.expand_dims(tf.to_float(classes), axis=-1)
        boxes = tf.concat([boxes[:, :5], classes], axis=-1)

        boxes_dict = dict()
        for cls in range(n_classes):
            mask = tf.equal(boxes[:, 5], cls)
            mask_shape = mask.get_shape()
            if mask_shape.ndims != 0:
                class_boxes = tf.boolean_mask(boxes, mask)
                
                boxes_coords, boxes_conf_scores, _ = tf.split(class_boxes,
                                                              [4, 1, -1],
                                                              axis=-1)
                boxes_conf_scores = tf.reshape(boxes_conf_scores, [-1])
                indices = tf.image.non_max_suppression(boxes_coords,
                                                       boxes_conf_scores,
                                                       max_output_size,
                                                       iou_thres)
                class_boxes = tf.gather(class_boxes, indices)
                boxes_dict[cls] = class_boxes[:, :5]

        boxed.append(boxes_dict)

    return boxed